In [25]:
## 네이버 영화평 감성 분석

In [26]:
!pip install konlpy

In [27]:
from google.colab import files

uploaded = files.upload()

Saving ratings_test.txt to ratings_test (1).txt


In [28]:
uploaded.keys()

dict_keys(['ratings_test.txt'])

In [29]:
import numpy as np 
import pandas as pd 

In [30]:
test_df = pd.read_csv(list(uploaded.keys())[0], sep='\t')
test_df.head(3)

,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0


In [31]:
uploaded = files.upload()
train_df = pd.read_csv(list(uploaded.keys())[0], sep='\t')
train_df.head(3)

Saving ratings_train.txt to ratings_train (2).txt


,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0


In [32]:
train_df.shape, test_df.shape 

((150000, 3), (50000, 3))

In [33]:
# 데이터 전처리

In [34]:
# 중복 여부 
train_df.document.nunique()

146182

In [35]:
train_df.drop_duplicates(subset=['document'], inplace=True)
train_df.shape

(146183, 3)

In [36]:
# 0과 1의 분포
train_df.label.value_counts()

0    73342
1    72841
Name: label, dtype: int64

In [37]:
# Null값 확인
train_df.isnull().sum()

id          0
document    1
label       0
dtype: int64

In [38]:
# Null값 제거
train_df = train_df.dropna(how='any')
train_df.shape

(146182, 3)

In [40]:
 ##- 테스트 데이터 셋에도 적용

In [42]:
test_df.document.nunique()

49157

In [43]:
test_df.drop_duplicates(subset=['document'], inplace=True)
test_df.shape

test_df.label.value_counts()

test_df.isnull().sum()

test_df = test_df.dropna(how='any')
test_df.shape

(49157, 3)

In [ ]:
### 텍스트 전처리 

In [44]:
# 한글 이외의 문자는 제거 
train_df['document'] = train_df['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "")
train_df.head()

,id,document,label
0,9976970,아 더빙 진짜 짜증나네요 목소리,0
1,3819312,흠포스터보고 초딩영화줄오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 솔직히 재미는 없다평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화스파이더맨에서 늙어보이기만 했던 커스틴 던...,1


In [45]:
# 모르는 상태에서는 못하고, 데이터를 유심히 살펴보아야 알 수 있음.
# 영문이나 숫자로만 되어 있는 문장은 데이터가 없어짐
train_df['document'].replace('', np.nan, inplace=True)
train_df.isnull().sum()

id            0
document    391
label         0
dtype: int64

In [46]:
train_df = train_df.dropna(how='any')
train_df.shape

(145791, 3)

In [ ]:
# 테스트 데이터 셋에도 해주기 

In [47]:
test_df['document'] = test_df['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "")
test_df.head()

,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,,0
2,8544678,뭐야 이 평점들은 나쁘진 않지만 점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임 돈주고 보기에는,0
4,6723715,만 아니었어도 별 다섯 개 줬을텐데 왜 로 나와서 제 심기를 불편하게 하죠,0


In [48]:
test_df['document'].replace('', np.nan, inplace=True)
test_df.isnull().sum()

id            0
document    162
label         0
dtype: int64

In [49]:
test_df = test_df.dropna(how='any')
test_df.shape

(48995, 3)

In [ ]:
### 한글 토큰화 

In [50]:
from konlpy.tag import Okt
okt = Okt()

In [52]:
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다','을']

In [53]:
okt.morphs('교도소 이야기구먼 솔직히 재미는 없다평점 조정', stem=True)

['교도소', '이야기', '구먼', '솔직하다', '재미', '는', '없다', '평점', '조정']

In [54]:
from tqdm import tqdm_notebook
X_train = []
for sentence in tqdm_notebook(train_df.document):
    morphs = okt.morphs(sentence, stem=True)   # 토큰화
    temp = ' '.join(word for word in morphs if not word in stopwords)  # 불용어 제거 
    X_train.append(temp)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


In [55]:
X_test = []
for sentence in tqdm_notebook(test_df.document):
    morphs = okt.morphs(sentence, stem=True)   # 토큰화
    temp = ' '.join(word for word in morphs if not word in stopwords)  # 불용어 제거 
    X_test.append(temp)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


In [58]:
y_train = train_df.label.values
y_test = test_df.label.values

In [59]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [60]:
tvect = TfidfVectorizer()
tvect.fit(X_train)
X_train_tvect = tvect.transform(X_train)
X_test_tvect = tvect.transform(X_test)


In [61]:
lr_clf = LogisticRegression()
lr_clf.fit(X_train_tvect, y_train)
pred = lr_clf.predict(X_test_tvect)
accuracy_score(y_test, pred)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.8305337279314216

In [ ]:
### 실제 테스트 

In [62]:
review1 = '진짜 개노잼이다.. 1편이랑 같은 감독맞나?러닝타임도 길어서 개지루함 ㄹㅇ'
review2 = '이런 사랑영화가 다시 나올 수 있을까?'

In [72]:
X_real_text = pd.DataFrame([review1]+[review2])
X_real_text

,0
0,진짜 개노잼이다.. 1편이랑 같은 감독맞나?러닝타임도 길어서 개지루함 ㄹㅇ
1,이런 사랑영화가 다시 나올 수 있을까?


In [76]:
import re
review1 = re.sub("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "", review1)
review1

'진짜 개노잼이다 편이랑 같은 감독맞나러닝타임도 길어서 개지루함 ㄹㅇ'

In [77]:
morphs = okt.morphs(review1, stem=True)   # 토큰화
review1 = ' '.join(word for word in morphs if not word in stopwords)  # 불용어 제거
review1

'진짜 개 노잼 이다 편이 랑 같다 감독 맞다 러닝 타임 기다 개 지루함 ㄹㅇ'

In [78]:
# review를 리스트로 만들어 주어야 함 
review1_tvect = tvect.transform([review1])

In [79]:
pred_rv1 = lr_clf.predict(review1_tvect)
pred_rv1

array([0])

In [81]:
review2 = re.sub("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "", review2)
morphs = okt.morphs(review2, stem=True)   # 토큰화
review2 = ' '.join(word for word in morphs if not word in stopwords)  # 불용어 제거
review2_tvect = tvect.transform([review2])
pred_rv2 = lr_clf.predict(review2_tvect)
pred_rv2

array([1])

In [87]:
reviews = [review1, review2]
sentiments = ['부정', '긍정']

In [88]:
review_list = []
for review in reviews:
    review = re.sub("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "", review)
    morphs = okt.morphs(review, stem=True)   # 토큰화
    temp = ' '.join(word for word in morphs if not word in stopwords)  # 불용어 제거
    review_list.append(temp)

In [89]:
review_list_tvect = tvect.transform(review_list)
pred_list = lr_clf.predict(review_list_tvect)

In [90]:
pred_list[0], pred_list[1]

(0, 1)

In [94]:
sentiments[pred_list[0]], sentiments[pred_list[1]]

('부정', '긍정')

In [95]:
for i in range(2):
    print(f'{reviews[i]} - {sentiments[pred_list[i]]}')

진짜 개 노잼 이다 편이 랑 같다 감독 맞다 러닝 타임 기다 개 지루함 ㄹㅇ - 부정
이렇다 사랑 영화 다시 나오다 수 있다 - 긍정


In [ ]:
## 임의 데이터로 테스트 

In [96]:
reviews = [
           '관람객 진짜 돈 주고 보면 안돼요^^허광환 볼꺼면 보셈 봐도 개 같아요 ㅠㅠ그냥 보지마세요',
           '남주 진짜 개존잘 .. 나의 소녀시대랑 장난스런 키스 다봤는데 개취로 이게 더 좋음! 결말은 반응 좀 갈리겠지만.. 유치하지 않고 나름 의미도 있는 거 같고 무엇보다 허광한 용안이 미쳤음 상견니 안봤는데 정주행각임 키스 개잘해',
           '이 영화는 오로지 허광한 얼굴뿐이다.',
           '하...평점 무슨 일입니까?... 저는 1점도 아깝습니다... 하... 살면서 본 영화 중에 가장 최악입니다...저는 사자가 최악일 줄 알았어요... 와 이거에 비하면... 하 스포일러라 차마 말은 못하지만 이건 정...'
]

In [97]:
review_list = []
for review in reviews:
    review = re.sub("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "", review)
    morphs = okt.morphs(review, stem=True)   # 토큰화
    temp = ' '.join(word for word in morphs if not word in stopwords)  # 불용어 제거
    review_list.append(temp)

In [98]:
review_list_tvect = tvect.transform(review_list)
pred_list = lr_clf.predict(review_list_tvect)

In [99]:
for i in range(2):
    print(f'{reviews[i]} - {sentiments[pred_list[i]]}')

관람객 진짜 돈 주고 보면 안돼요^^허광환 볼꺼면 보셈 봐도 개 같아요 ㅠㅠ그냥 보지마세요 - 부정
남주 진짜 개존잘 .. 나의 소녀시대랑 장난스런 키스 다봤는데 개취로 이게 더 좋음! 결말은 반응 좀 갈리겠지만.. 유치하지 않고 나름 의미도 있는 거 같고 무엇보다 허광한 용안이 미쳤음 상견니 안봤는데 정주행각임 키스 개잘해 - 부정
